In [17]:
import pandas as pd
import numpy as np
import re
import os
import zipfile
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Embedding, Flatten
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras_tuner import RandomSearch
import tensorflow as tf
import numpy as np

In [18]:
# 解压 cleaned_lyrics.zip 文件
with zipfile.ZipFile('cleaned_lyrics.zip', 'r') as zip_ref:
    zip_ref.extractall('cleaned_lyrics')

# 获取所有歌词文件的路径
lyrics_files = {os.path.splitext(f)[0]: os.path.join('cleaned_lyrics', f) for f in os.listdir('cleaned_lyrics')}

# 读取 filtered_dataset.csv 文件
data = pd.read_csv('filtered_dataset.csv')

def read_lyrics(record_id):
    file_path = lyrics_files.get(str(record_id))
    if file_path and os.path.exists(file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    return ''

# 读取歌词并添加到数据框中
data['lyrics'] = data['record_id'].apply(read_lyrics)


In [19]:
data

,Unnamed: 0.1,record_id,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,...,instrumentalness,liveness,valence,tempo,time_signature,track_genre,valence_bin,energy_bin,danceability_bin,lyrics
0,1,1,740,1T7Tqsfkz0Ntbwta2hHebY,Days N Daze,Rogue Taxidermy,Fate of a Coward,28,175046,False,...,0.000000,0.3510,0.9430,123.094,4,acoustic,2,1.0,1,mind seeping darkness pulse growing weaker mom...
1,3,3,513,27hFQQS3cVUmIK3ser5bpu,Colin & Caroline,More Than Gravity,More Than Gravity,34,266078,False,...,0.000039,0.1060,0.2170,110.969,4,acoustic,0,1.0,1,simple explanation things feel one word tell t...
2,4,4,136,2PIlBukQ6limukVR8Ubb5o,Gabrielle Aplin,English Rain,Please Don't Say You Love Me,59,181400,False,...,0.000000,0.1080,0.3210,85.994,4,acoustic,0,1.0,1,summer comes winter fades not pressure not cha...
3,5,5,899,4uPvXmXGjYOOqhbRMmS9XU,Grace Petrie,Queer As Folk,Northbound,26,270920,False,...,0.000000,0.1410,0.5860,125.642,4,acoustic,1,2.0,1,gone lonesome road goes forever espresso shot ...
4,9,9,549,2saK0E712wIB3Gf8QLuFYX,Get Dead,Dancing with the Curse,Nickel Plated,29,136815,True,...,0.000000,0.6540,0.4000,179.920,4,acoustic,1,2.0,1,nickel plated tooth briefcase hold breath poli...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5896,10802,10802,113500,2kDe0QRaCBKIh8QY64GDvK,Bethel Music;Jenn Johnson,We Will Not Be Shaken (Live),In Over My Head (Crash Over Me) - Live,49,298681,False,...,0.029400,0.0833,0.0896,141.800,3,world-music,0,1.0,1,come place life full satisfied longing feel he...
5897,10803,10803,113774,4EQYur0tRZpHbQJxgCRy4Q,Michael W. Smith,Worship,"More Love, More Power - Live",39,310293,False,...,0.000094,0.3440,0.1510,142.075,4,world-music,0,0.0,0,love power life love power life worship heart ...
5898,10805,10805,113108,6CW9qtzZpHZ3o39BYlpU0x,Bethel Music;Bethany Wohrle,Living Hope,Living Hope,53,406346,False,...,0.000000,0.5290,0.1850,143.957,4,world-music,0,2.0,1,great chasm lay high mountain climb desperatio...
5899,10807,10807,113056,2elEVvWjPZltkotzcCwKvM,Kari Jobe;Cody Carnes;Elevation Worship,The Blessing (Live),The Blessing - Live,61,514665,False,...,0.000000,0.2220,0.1970,140.015,4,world-music,0,1.0,1,lord bless keep make face shine upon gracious ...


In [20]:
len(lyrics_files)

5901

In [21]:
data

,Unnamed: 0.1,record_id,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,...,instrumentalness,liveness,valence,tempo,time_signature,track_genre,valence_bin,energy_bin,danceability_bin,lyrics
0,1,1,740,1T7Tqsfkz0Ntbwta2hHebY,Days N Daze,Rogue Taxidermy,Fate of a Coward,28,175046,False,...,0.000000,0.3510,0.9430,123.094,4,acoustic,2,1.0,1,mind seeping darkness pulse growing weaker mom...
1,3,3,513,27hFQQS3cVUmIK3ser5bpu,Colin & Caroline,More Than Gravity,More Than Gravity,34,266078,False,...,0.000039,0.1060,0.2170,110.969,4,acoustic,0,1.0,1,simple explanation things feel one word tell t...
2,4,4,136,2PIlBukQ6limukVR8Ubb5o,Gabrielle Aplin,English Rain,Please Don't Say You Love Me,59,181400,False,...,0.000000,0.1080,0.3210,85.994,4,acoustic,0,1.0,1,summer comes winter fades not pressure not cha...
3,5,5,899,4uPvXmXGjYOOqhbRMmS9XU,Grace Petrie,Queer As Folk,Northbound,26,270920,False,...,0.000000,0.1410,0.5860,125.642,4,acoustic,1,2.0,1,gone lonesome road goes forever espresso shot ...
4,9,9,549,2saK0E712wIB3Gf8QLuFYX,Get Dead,Dancing with the Curse,Nickel Plated,29,136815,True,...,0.000000,0.6540,0.4000,179.920,4,acoustic,1,2.0,1,nickel plated tooth briefcase hold breath poli...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5896,10802,10802,113500,2kDe0QRaCBKIh8QY64GDvK,Bethel Music;Jenn Johnson,We Will Not Be Shaken (Live),In Over My Head (Crash Over Me) - Live,49,298681,False,...,0.029400,0.0833,0.0896,141.800,3,world-music,0,1.0,1,come place life full satisfied longing feel he...
5897,10803,10803,113774,4EQYur0tRZpHbQJxgCRy4Q,Michael W. Smith,Worship,"More Love, More Power - Live",39,310293,False,...,0.000094,0.3440,0.1510,142.075,4,world-music,0,0.0,0,love power life love power life worship heart ...
5898,10805,10805,113108,6CW9qtzZpHZ3o39BYlpU0x,Bethel Music;Bethany Wohrle,Living Hope,Living Hope,53,406346,False,...,0.000000,0.5290,0.1850,143.957,4,world-music,0,2.0,1,great chasm lay high mountain climb desperatio...
5899,10807,10807,113056,2elEVvWjPZltkotzcCwKvM,Kari Jobe;Cody Carnes;Elevation Worship,The Blessing (Live),The Blessing - Live,61,514665,False,...,0.000000,0.2220,0.1970,140.015,4,world-music,0,1.0,1,lord bless keep make face shine upon gracious ...


In [22]:
# 使用 Tokenizer 处理文本
max_words = 5000
max_len = 100

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data['lyrics'])
sequences = tokenizer.texts_to_sequences(data['lyrics'])
X_lyrics = pad_sequences(sequences, maxlen=max_len)

# 准备标签
y_valence = to_categorical(data['valence_bin'].values)
y_energy = to_categorical(data['energy_bin'].values)
y_danceability = to_categorical(data['danceability_bin'].values)

# 拆分数据集
X_train_val, X_test, y_train_val_valence, y_test_valence, y_train_val_energy, y_test_energy, y_train_val_danceability, y_test_danceability = train_test_split(
    X_lyrics, y_valence, y_energy, y_danceability, test_size=0.2, random_state=42)

X_train, X_val, y_train_valence, y_val_valence, y_train_energy, y_val_energy, y_train_danceability, y_val_danceability = train_test_split(
    X_train_val, y_train_val_valence, y_train_val_energy, y_train_val_danceability, test_size=0.2, random_state=42)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/utils/np_utils.py:62: RuntimeWarning: invalid value encountered in cast
  y = np.array(y, dtype="int")


In [23]:
X_train

array([[ 229,  582,  229, ...,  146,  119,   31],
       [   1,    7,  903, ...,   11,  142,   31],
       [ 729,    1,  342, ...,   87,  694,   31],
       ...,
       [  39,    3,   16, ...,   45,  150,   45],
       [ 155,  208,  952, ...,   70,  613,   31],
       [2505,   54,    7, ...,  299,  428,   31]], dtype=int32)

In [24]:
from keras_tuner import RandomSearch, HyperParameters, Objective
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Embedding, Flatten
from keras.optimizers import Adam, RMSprop, SGD
from keras.callbacks import EarlyStopping


# 构建模型函数
def build_model(hp):
    inputs = Input(shape=(max_len,))
    x = Embedding(input_dim=hp.Int('input_dim', min_value=1000, max_value=10000, step=1000),
                  output_dim=hp.Int('output_dim', min_value=32, max_value=128, step=32),
                  input_length=max_len)(inputs)
    x = Flatten()(x)

    num_layers = hp.Int('num_layers', min_value=1, max_value=5, step=1)
    for i in range(num_layers):
        if i == 0:
            x = Dense(units=hp.Int(f'units_layer{i+1}', min_value=32, max_value=512, step=32), activation='relu')(x)
        else:
            x = Dense(units=hp.Int(f'units_layer{i+1}', min_value=32, max_value=512, step=32), activation='relu')(x)
        x = Dropout(rate=hp.Float(f'dropout_layer{i+1}', min_value=0.0, max_value=0.5, step=0.1))(x)

    x = Dense(units=hp.Int('units_final', min_value=32, max_value=512, step=32),
              activation='relu',
              kernel_regularizer=tf.keras.regularizers.l2(hp.Choice('l2_regularization', values=[0.0, 1e-4, 1e-3])),
              kernel_initializer=hp.Choice('kernel_initializer', values=['glorot_uniform', 'he_normal']))(x)

    
    output_valence = Dense(y_valence.shape[1], activation='softmax', name='valence_output')(x)
    output_energy = Dense(y_energy.shape[1], activation='softmax', name='energy_output')(x)
    output_danceability = Dense(y_danceability.shape[1], activation='softmax', name='danceability_output')(x)
    
    model = Model(inputs=inputs, outputs=[output_valence, output_energy, output_danceability])

    optimizer_choice = hp.Choice('optimizer', values=['adam', 'rmsprop', 'sgd'])
    learning_rate = hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5])

    if optimizer_choice == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer_choice == 'rmsprop':
        optimizer = RMSprop(learning_rate=learning_rate)
    else:
        optimizer = SGD(learning_rate=learning_rate)

    model.compile(optimizer=optimizer,
                  loss={'valence_output': 'categorical_crossentropy', 
                        'energy_output': 'categorical_crossentropy', 
                        'danceability_output': 'categorical_crossentropy'},
                  metrics={'valence_output': 'accuracy', 
                           'energy_output': 'accuracy', 
                           'danceability_output': 'accuracy'})
    return model

# 超参数调优
tuner = RandomSearch(
    build_model,
    objective=Objective('val_valence_output_accuracy', direction='max'),
    max_trials=10,
    executions_per_trial=1,
    directory='tuner_dir',
    project_name='mood_detection5'
)

# 启动调优过程
tuner.search(X_train, [y_train_valence, y_train_energy, y_train_danceability], 
             epochs=20, 
             validation_data=(X_val, [y_val_valence, y_val_energy, y_val_danceability]), 
             callbacks=[EarlyStopping(patience=3)])

# 获取最佳模型
best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_hyperparameters.values)

# 评估模型

loss, valence_output_loss, energy_output_loss, danceability_output_loss, accuracy_valence, accuracy_energy, accuracy_danceability = best_model.evaluate(X_test, [y_test_valence, y_test_energy, y_test_danceability])
print(f'Test Loss: {loss}, valence_output_loss: {valence_output_loss}, energy_output_loss: {energy_output_loss}, danceability_output_loss: {danceability_output_loss}, Test Accuracy Valence: {accuracy_valence}, Test Accuracy Energy: {accuracy_energy}, Test Accuracy Danceability: {accuracy_danceability}')

Trial 10 Complete [00h 00m 01s]

Best val_valence_output_accuracy So Far: 0.6514830589294434
Total elapsed time: 00h 00m 47s
{'input_dim': 7000, 'output_dim': 32, 'num_layers': 2, 'units_layer1': 320, 'dropout_layer1': 0.30000000000000004, 'units_final': 384, 'l2_regularization': 0.0, 'kernel_initializer': 'glorot_uniform', 'optimizer': 'adam', 'learning_rate': 0.001, 'units_layer2': 32, 'dropout_layer2': 0.0}
37/37 [==============================] - 0s 2ms/step - loss: 3.4744 - valence_output_loss: 1.2052 - energy_output_loss: 1.2697 - danceability_output_loss: 0.9995 - valence_output_accuracy: 0.6274 - energy_output_accuracy: 0.6749 - danceability_output_accuracy: 0.7341
Test Loss: 3.4744436740875244, valence_output_loss: 1.2052104473114014, energy_output_loss: 1.2696882486343384, danceability_output_loss: 0.9995446801185608, Test Accuracy Valence: 0.6274343729019165, Test Accuracy Energy: 0.6748518347740173, Test Accuracy Danceability: 0.7341236472129822


In [25]:
metrics = best_model.evaluate(X_test, [y_test_valence, y_test_energy, y_test_danceability])
#print(f'Test Loss: {loss}, Test Accuracy Valence: {accuracy_valence}, Test Accuracy Energy: {accuracy_energy}, Test Accuracy Danceability: {accuracy_danceability}')
print(metrics)

37/37 [==============================] - 0s 2ms/step - loss: 3.4744 - valence_output_loss: 1.2052 - energy_output_loss: 1.2697 - danceability_output_loss: 0.9995 - valence_output_accuracy: 0.6274 - energy_output_accuracy: 0.6749 - danceability_output_accuracy: 0.7341
[3.4744436740875244, 1.2052104473114014, 1.2696882486343384, 0.9995446801185608, 0.6274343729019165, 0.6748518347740173, 0.7341236472129822]
